In [11]:
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Layer
from keras import backend as K
import numpy as np

In [12]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [13]:
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_train = x_train / 255
x_test = x_test / 255

In [14]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [15]:
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                              shape=(int(input_shape[1]), self.units),
                              initializer='uniform',
                              trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [16]:
model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh',
input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(RBFLayer(10, 0.5))

In [17]:
model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, verbose=1,
validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0219 - accuracy: 0.9129 - val_loss: 0.0101 - val_accuracy: 0.9741
Epoch 2/5
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0089 - accuracy: 0.9741 - val_loss: 0.0075 - val_accuracy: 0.9748
Epoch 3/5
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0059 - accuracy: 0.9788 - val_loss: 0.0046 - val_accuracy: 0.9814
Epoch 4/5
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0043 - accuracy: 0.9818 - val_loss: 0.0038 - val_accuracy: 0.9837
Epoch 5/5
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0036 - accuracy: 0.9844 - val_loss: 0.0034 - val_accuracy: 0.9858


In [18]:
score = model.evaluate(x_test, y_test)
print('테스트 정확도 :', score[1])

313/313 [==============================] - 3s 8ms/step - loss: 0.0034 - accuracy: 0.9858
테스트 정확도 : 0.98580002784729
